In [1]:
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
import warnings
warnings.filterwarnings("ignore")

# Classifiers
from catboost import CatBoostClassifier

# Model selection
from sklearn.model_selection import StratifiedKFold, PredefinedSplit

# Metrics
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import make_scorer

# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer
from time import time

In [2]:
tr_df = pd.read_csv('../../../data/feature/cba_train.csv')

In [3]:
tr_df.head()

,Gender,Age,Ethnicity,Educational_Level,Income,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Required_Car_Parking,Use_Promotion,Room_Rate,month_Expected_checkin,dayofweek_Expected_checkin,stay,booking_to_checkin,total_participants,Reservation_Status
0,0.002971,40,0.010376,-0.001419,0.007453,-0.444474,0.010791,0.003794,0.022095,0.005952,0.004285,218,7,2,1,41,4,1
1,0.002971,49,0.010376,0.006746,-0.012088,-0.444474,0.010791,0.003794,-0.069188,0.005952,-0.013100,185,7,2,1,36,6,1
2,0.002971,42,-0.017315,-0.001419,0.007453,-0.444474,0.010791,0.003794,0.022095,0.005952,-0.013100,119,7,3,4,3,6,1
3,-0.003235,25,0.014371,-0.003171,-0.012970,-0.444474,0.010791,0.003794,-0.069188,0.005952,0.004285,144,7,3,1,12,7,1
4,0.002971,62,0.010376,0.002616,0.012407,-0.444474,0.010791,0.003794,0.022095,-0.014993,0.004285,242,7,4,1,13,2,1


In [4]:
tr_df.iloc[:, -1].unique()

array([1, 2, 3], dtype=int64)

In [4]:
#tr_df.columns
cat_inds = [9,2,3,4,5,6,9,10,11,12,13,14,15]


In [5]:
def report_perf(optimizer, X, y,title, callbacks=None):
    """
    A wrapper for measuring time and performances of different optmizers
    
    optimizer = a sklearn or a skopt optimizer
    X = the training set 
    y = our target
    title = a string label for the experiment
    """
    start = time()
    if callbacks:
        optimizer.fit(X, y,callback=callbacks)
    else:
        optimizer.fit(X, y)
    d=pd.DataFrame(optimizer.cv_results_)
    best_score = optimizer.best_score_
    best_score_std = d.iloc[optimizer.best_index_].std_test_score
    best_params = optimizer.best_params_
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           +u"\u00B1"+" %.3f") % (time() - start, 
                                  len(optimizer.cv_results_['params']),
                                  best_score,
                                  best_score_std))    
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

In [6]:
val_df = pd.read_csv('../../../data/feature/cba_validation.csv').sample(frac=1)
cv_size = val_df.shape[0]//2
full_stack = [tr_df, val_df.iloc[:cv_size, :]]
full_df = pd.concat(full_stack)

ind_list = [-1,]*tr_df.shape[0]+[0,]*cv_size
prd = PredefinedSplit(ind_list)



In [7]:
score = make_scorer(f1_score, average='macro')
# score = make_scorer(accuracy_score)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=443)

In [8]:
clf = CatBoostClassifier(thread_count=3,
                         loss_function='MultiClass',
                         od_type = 'Iter',
                         task_type="GPU",
                         verbose= False
                        )
# cat_features=cat_inds,

In [9]:
search_spaces = {'iterations': Integer(10, 1000),
                 'depth': Integer(1, 8),
                 'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                 'random_strength': Real(1e-9, 10, 'log-uniform'),
                 'bagging_temperature': Real(0.0, 1.0),
                 'border_count': Integer(1, 255),
                 'l2_leaf_reg': Integer(2, 30)}

In [10]:
opt = BayesSearchCV(clf,
                    search_spaces,
                    scoring=score,
                    cv=skf,
                    n_iter=100,
                    n_jobs=1,  # use just 1 job with CatBoost in order to avoid segmentation fault
                    return_train_score=False,
                    random_state=443)
# optimizer_kwargs={'base_estimator': 'GP'},
#                     refit=True,

In [16]:

best_params = report_perf(opt, tr_df.iloc[:, :-1], tr_df.iloc[:, -1],'CatBoost', 
                          callbacks=[VerboseCallback(100), 
                                     DeadlineStopper(60*10)])

Iteration No: 1 started. Searching for the next optimal point.
Iteration No: 1 ended. Search finished for the next optimal point.
Time taken: 5.8510
Function value obtained: -0.2905
Current minimum: -0.2905
Iteration No: 2 started. Searching for the next optimal point.


KeyboardInterrupt: 

In [12]:
#val_df = pd.read_csv('../../../data/feature/cb_validation.csv')
X_val = val_df.iloc[cv_size:, :-1]
y_val = val_df.iloc[cv_size:, -1]

In [13]:
y_val_pred = opt.predict(X_val)

In [14]:
print(f1_score(y_val, y_val_pred, average='macro'))

0.3325208857519097


In [25]:
pred_df = pd.DataFrame()
pred_df['true'] = y_val
pred_df['prediction'] = y_val_pred


In [26]:
pred_df.head()

,true,prediction
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [15]:
ts_df = pd.read_csv('../../../data/feature/cb_test.csv')
ts_df.head()


,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Meal_Type,Visted_Previously,...,Deposit_type,Booking_channel,Required_Car_Parking,Use_Promotion,Room_Rate,month_Expected_checkin,dayofweek_Expected_checkin,stay,booking_to_checkin,total_participants
0,62931593,0,52,0,0,3,2,0,2,0,...,0,2,0,0,153,11,4,1,21,6
1,70586099,0,47,0,0,3,1,1,1,0,...,0,0,1,1,210,11,4,1,104,3
2,4230648,0,28,3,0,0,1,0,0,0,...,0,1,1,0,117,4,4,3,20,4
3,25192322,0,65,1,3,3,2,1,1,0,...,0,0,0,0,107,11,4,2,182,6
4,80931528,1,45,2,2,3,2,0,0,0,...,1,1,1,1,119,11,4,2,18,4


In [16]:
preds = opt.predict(ts_df.iloc[:, 1:])

In [17]:
sub_df = pd.DataFrame()
sub_df['Reservation-id'] = ts_df.iloc[:, 0]
sub_df['Reservation_status'] = preds

In [19]:
sub_df.to_csv('submission2.csv', index=False)

In [23]:
val_df = pd.read_csv('../../../data/feature/cb_validation.csv')
tr_df = pd.read_csv('../../../data/feature/cb_train.csv')
print(f'train size : {tr_df.shape[0]}, validation size : {val_df.shape[0]}')


train size : 27499, validation size : 2749


In [12]:
full_stack = [tr_df, val_df]
full_df = pd.concat(full_stack)

In [15]:
full_df.sample(frac=1).head()
full_df.shape

,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Meal_Type,Visted_Previously,Previous_Cancellations,...,Booking_channel,Required_Car_Parking,Use_Promotion,Room_Rate,month_Expected_checkin,dayofweek_Expected_checkin,stay,booking_to_checkin,total_participants,Reservation_Status
24781,1,32,0,2,3,3,0,2,0,0,...,2,0,0,173,6,4,1,150,5,0
8244,0,54,1,2,1,2,2,0,0,0,...,0,0,0,134,6,3,1,214,2,0
13519,0,18,3,0,0,3,2,2,1,0,...,0,0,0,162,5,4,3,100,4,0
21455,0,49,3,0,3,2,2,0,0,0,...,0,0,0,213,2,0,1,33,4,2
13450,1,52,2,0,0,1,2,1,0,0,...,0,0,0,239,5,0,2,60,5,0


(30248, 21)

In [16]:
ind_list = [-1,]*tr_df.shape[0]+[0,]*val_df.shape[0]

In [17]:
prd = PredefinedSplit(ind_list)

In [28]:
val_df.shape[0]/2

1374.5